# DMD Project Database Querying

In [0]:
!pip install --upgrade firebase-admin

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import auth
import pandas as pd
from pprint import pprint
from IPython.display import display
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 1000)

## Establish connection

In [2]:
def establish_creds():
    # Use a service account
    cred = credentials.Certificate('admin_cred.json')
    firebase_admin.initialize_app(cred)
establish_creds()

In [3]:
db = firestore.client()

In [4]:
def print_doc(doc):
    d = doc.to_dict()
    df = pd.DataFrame([d], index=[doc.id])
    display(df)

def print_docs(docs):
    for doc in docs:
        print_doc(doc)

def print_dict(doc):
    pprint(doc.to_dict())

def print_dicts(docs):
    for doc in docs:
        print_dict(doc)

## read the Employees and Patients collections

In [5]:
users_ref = db.collection("employees").limit(5)
docs = users_ref.stream()
print_docs(docs)

,address,contactNumber,gender,name,role,salary
0DKu7oIWmNibuW80AXy1,"18459 Johnson Haven\nSouth Stephanie, NY 70923",797.068.2595x62393,M,James Barnes,nurse,17540


,address,contactNumber,gender,name,role,salary,status
0Dnw3tOd3FrnPRUNZbUu,"300 Brown Mews Apt. 012\nNorth Ryan, ME 90964",898.180.7691,F,Amanda Murphy,doctor,31600,permanent


,address,contactNumber,gender,name,role,salary,status
0FJVjirro8ehObVMp4NB,"7798 Parker Turnpike Apt. 223\nWest Jennifer, AL 32862",(138)796-0864,M,James Scott,doctor,27210,permanent


,address,contactNumber,gender,name,role,salary
0Q2uajUrTnjy7GrnyjsL,"548 Walter Springs\nNorth Mark, AK 99647",+1-379-356-2954x8517,F,Mrs. Pamela Watson,pharmacist,16590


,address,contactNumber,gender,name,role,salary
0QrhT2RErKsvSVzjDzj9,"821 Wheeler Trafficway\nLake Jennifer, AK 18945",006-382-7945,F,Vanessa Knight,nurse,19450


In [6]:
def get_docs(db, collection, limit=5):
    return db.collection(collection).limit(limit).stream()

In [7]:
patients = get_docs(db, 'patients')
print_dicts(patients)

{'address': '3978 Charles Point Apt. 831\nMichaelton, ID 49992',
 'contactNumber': '+1-161-220-3424x339',
 'email': 'idixon@yahoo.com',
 'gender': 'M',
 'medHistory': [{'date': DatetimeWithNanoseconds(2018, 2, 25, 20, 8, 30, tzinfo=<UTC>),
                 'note': 'Debate scene line only than hard receive. Congress '
                         'charge agree right treatment action much. Without '
                         'risk remember laugh free spring.',
                 'prescription': None},
                {'date': DatetimeWithNanoseconds(2018, 4, 13, 6, 1, 30, tzinfo=<UTC>),
                 'note': 'South read specific south. Film fine family career.',
                 'prescription': None},
                {'date': DatetimeWithNanoseconds(2018, 6, 6, 7, 46, 21, tzinfo=<UTC>),
                 'note': 'Common stop us door able human red. Sign democratic '
                         'former value commercial money while.',
                 'prescription': None}],
 'name': 'Philip Hill'

## add a new employee (also register him as a user)

In [8]:
new_employee = {
    'address': 'Innopolis',
    'contactNumber': '+79731577181',
    'email': 'r.maksudov@innopolis.ru',
    'gender': 'M',
    'name': 'Rishat Maksudov',
    'role': 'Nurse',
    'salary': 12000,
}

In [9]:
def register_user(collection, data, password):
    ref = db.collection(collection).document()
    ref.set(data)
    user = auth.create_user(email=data['email'], phone_number=data['contactNumber'], password=password)
    print('Sucessfully created new user: {0}'.format(user.uid))
    return ref, user.uid

In [10]:
employee_ref, employee_uid = register_user('employees', new_employee, '1234567890aA')

Sucessfully created new user: V2EvWvQmpRTe21fAnFSSAPgn30l2


**Check it!**

**See Sign-in Operation on Website Demo**

## update information of the employee

In [11]:
def edit_profile(ref, data):
    employee_ref.update(data)

In [12]:
edit_profile(employee_ref, {
      'role': 'doctor',
      'status': 'intern',
      'salary': 18000,
  })

In [13]:
print_doc(employee_ref.get())

,address,contactNumber,email,gender,name,role,salary,status
ALfUjLrj1evKmJtb227P,Innopolis,+79731577181,r.maksudov@innopolis.ru,M,Rishat Maksudov,doctor,18000,intern


## delete the user as well as document from collection of Employees

In [14]:
employee_ref.delete()
auth.delete_user(employee_uid)
print('Successfully deleted user')

Successfully deleted user


## get a chat of a patient with a doctor

In [21]:
doctor = db.collection('employees').document('GqzbDbOpk9YvbWIYg6Kb')
patient = db.collection('patients').document('zEVeX6F0w0QdWf5tUAfs')

In [22]:
def get_chat_messages(doctor, patient):
    docs = db.collection('chats').where('doctor', '==', doctor).where('patient', '==', patient).stream()
    for doc in docs:
        print_dict(doc)

In [23]:
get_chat_messages(doctor, patient)

## get a list of medicines with expiration date due to some date/time

In [25]:
def get_medicines_by_exp_date(exp_date):
    return db.collection('medicines').limit(10).where('expDate', '<', exp_date).stream()

In [26]:
date = datetime.now() + timedelta(days=0, hours=-5)
query = get_medicines_by_exp_date(date)
print_docs(query)

,expDate,name,price,quantity,sold
iAAzwZ75hSCUuJhEd09U,2019-04-23 21:41:36+00:00,Zinerit,270,89 unit(s),True


,expDate,name,price,quantity,sold
PN0cfTKBXT6DUx6pNv3z,2019-04-24 00:19:43+00:00,Zantak,175,17 unit(s),True


,expDate,name,price,quantity,sold
ey3HJEprSRXPv6GNJ4TZ,2019-04-24 03:06:35+00:00,Itrazol,40,66 ml,True


,expDate,name,price,quantity,sold
wZXDMmNTdyKfKf3NCPM1,2019-04-24 03:08:36+00:00,Itrazol,40,97 ml,True


## more complicated stuff
 **Imitate the prescription writing and patient's action to buy the medicines using this prescription**

In [30]:
def get_one_doctor(db):
    docs = db.collection('employees').limit(20).where('role', '==', 'doctor').limit(1).stream()
    for doc in docs:
        return doc

def get_one_patient(db):
    docs = db.collection('patients').limit(1).stream()
    for doc in docs:
        return doc

def get_not_sold_medicines(db, limit=3):
    docs = db.collection('medicines').limit(40).where('sold', '==', False).limit(limit).stream()
    return docs

In [28]:
def write_a_prescription(db, doctor, patient, description, med_list):
    date = datetime.now()
    data = {
      'doctor': doctor,
      'patient': patient,
      'date': date,
      'description': description,
      'medList': med_list,
    }
    ref = db.collection('prescriptions').document()
    ref.set(data)
    print("Successfully wrote a prescription")
    return ref

**get a doctor and a patient as well as a list of medicines, write a prescription**

In [31]:
doctor = get_one_doctor(db).reference
patient = get_one_patient(db).reference
med_list = [med.reference for med in get_not_sold_medicines(db)]

prescr_ref = write_a_prescription(db, doctor, patient, "Description", med_list)

Successfully wrote a prescription


In [32]:
print_dict(prescr_ref.get())

{'date': DatetimeWithNanoseconds(2019, 4, 24, 10, 36, 19, 218490, tzinfo=<UTC>),
 'description': 'Description',
 'doctor': <google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac951af28>,
 'medList': [<google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac951a5c0>,
 'patient': <google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac951aba8>}


In [33]:
class Bill:
    def __init__(self, db: firestore, med_refs, buyer_ref):
        self.db = db
        self.collection = 'bills'
        self.date = datetime.now()
        self.buyer = buyer_ref
        self.medList = med_refs
        self.ref = None
        self.id = None

    def to_dict(self):
        return {
            u'date': self.date,
            u'buyer': self.buyer,
            u'medList': self.medList,
        }

    def save(self):
        self.ref = self.db.collection(self.collection).document()
        self.id = self.ref.id
        self.ref.set(self.to_dict())
        return self.ref

**functionality to buy medicines**

In [34]:
def find_medicines(db, med_list):
    med_docs = []
    for ref in med_list:
        med = ref.get().to_dict()
        docs = db.collection('medicines').limit(50).where('name', '==', med['name']).where('sold', '==', False).limit(1).stream()
        for doc in docs:
            med_docs.append(doc.reference)
            break
    return med_docs

def buy_medicines(db, prescription):
    # create a bill
    prescr_doc = prescription.get().to_dict()
    buyer = prescr_doc['patient']
    med_docs = find_medicines(db, prescr_doc['medList'])
    bill = Bill(db, med_docs, prescr_doc['patient'])
    bill.save()
  
    # update bills list of prescription
    prescription.update({'bills': [bill.ref]})

    # check medicines as sold
    for med_doc in med_docs:
        med_doc.update({'sold': True})

    print("The patient successfully bought medicines")
    return bill.ref, med_docs

**imitate the patient buying the medicines listed in a prescription**

In [35]:
bill_ref, med_refs = buy_medicines(db, prescr_ref)

The patient successfully bought medicines


In [36]:
print_dict(bill_ref.get())
print()
print_dicts([doc.get() for doc in med_refs])

{'buyer': <google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac9289630>,
 'date': DatetimeWithNanoseconds(2019, 4, 24, 10, 37, 27, 157508, tzinfo=<UTC>),
 'medList': [<google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac9289470>,
             <google.cloud.firestore_v1.document.DocumentReference object at 0x7f8ac9289358>]}

{'expDate': DatetimeWithNanoseconds(2019, 5, 12, 16, 32, 15, tzinfo=<UTC>),
 'name': 'Zivox',
 'price': 220,
 'quantity': '62 unit(s)',
 'sold': True}
{'expDate': DatetimeWithNanoseconds(2019, 5, 25, 21, 0, tzinfo=<UTC>),
 'name': 'Azue',
 'price': '120',
 'quantity': '25ml',
 'sold': True}
{'expDate': DatetimeWithNanoseconds(2019, 7, 21, 5, 23, 43, tzinfo=<UTC>),
 'name': 'Citramon',
 'price': 20,
 'quantity': '97 unit(s)',
 'sold': True}


In [37]:
for doc in med_refs:
    doc.update({'sold': False})